In [1]:
import clickhouse_connect
import pandas as pd
import datetime
import csv

In [2]:
client = clickhouse_connect.get_client(host='localhost', username='default', password='')

In [3]:
client.query('''DROP TABLE IF EXISTS session''')
client.query('''DROP TABLE IF EXISTS mat_view;''')

client.command('''
    CREATE TABLE session (
        id VARCHAR(50),
        time DateTime,
        action VARCHAR(30),
    )
    ENGINE MergeTree()
    PRIMARY KEY (id, time);
''')

client.query('''
    CREATE MATERIALIZED VIEW mat_view ENGINE = MergeTree PRIMARY KEY (count_new_sessions)
    AS
    SELECT COUNT(*) count_new_sessions
    FROM (SELECT *, time - lagInFrame(time) OVER (PARTITION BY id ORDER BY time) dif_in_minutes
                   FROM session)
    WHERE dif_in_minutes > 15;
''')

In [4]:
with open(f'test_task_2.csv', 'r', encoding='utf-8') as csv_file:
    csv_reader = csv.reader(csv_file)
    data = [(row[0], datetime.datetime.strptime(row[1], '%Y-%m-%d %H:%M:%S'), row[2]) for row in csv_reader]
    print(len(data))
    client.insert('session', data)

26000


In [5]:
result = client.query('''SELECT * FROM session''')
df = pd.DataFrame(columns=result.column_names, data=result.result_rows)
df

,id,time,action
0,1690618913096-4258758771416604464,2024-01-22 12:38:35+03:00,launch
1,1690618913096-4258758771416604464,2024-01-22 12:38:59+03:00,workshop_build
2,1690618913096-4258758771416604464,2024-01-22 12:38:59+03:00,workshop_build
3,1690618913096-4258758771416604464,2024-01-22 12:38:59+03:00,launch
4,1690618913096-4258758771416604464,2024-01-22 12:39:00+03:00,lp_start
...,...,...,...
25995,1705944698122-7121932,2024-01-22 23:26:25+03:00,rv_show
25996,1705944698122-7121932,2024-01-22 23:26:27+03:00,rv_click
25997,1705944698122-7121932,2024-01-22 23:26:29+03:00,rv_start
25998,1705944698122-7121932,2024-01-22 23:49:42+03:00,rv_show


In [6]:
# Checking if Materialized view works
_ = client.query('''SELECT * FROM mat_view''')
pd.DataFrame(columns=_.column_names, data=_.result_rows)

,count_new_sessions
0,5082
